# Part I. ETL Pipeline for Pre-Processing the Files

## PLEASE RUN THE FOLLOWING CODE FOR PRE-PROCESSING THE FILES

#### Import Python packages 

In [1]:
# Import Python packages 
import pandas as pd
import cassandra
import re
import os
import glob
import numpy as np
import json
import csv

#### Creating list of filepaths to process original event csv data files

In [11]:
# checking your current working directory
print(os.getcwd())

# Get your current folder and subfolder event data
filepath = os.getcwd() + '/event_data'

file_path_list = []

# Create a for loop to create a list of files and collect each filepath
for root, dirs, files in os.walk(filepath):
# join the file path and roots with the subdirectories using glob
    new_paths = glob.glob(os.path.join(root,'*'))
    
    # make sure that no additional files (e.g., ipynb checkpoints) are caught in this list.
    for path in new_paths:
        if path.endswith('-events.csv'):
            file_path_list.append(path)
            
print(file_path_list)

/home/workspace
['/home/workspace/event_data/2018-11-27-events.csv', '/home/workspace/event_data/2018-11-04-events.csv', '/home/workspace/event_data/2018-11-07-events.csv', '/home/workspace/event_data/2018-11-09-events.csv', '/home/workspace/event_data/2018-11-19-events.csv', '/home/workspace/event_data/2018-11-05-events.csv', '/home/workspace/event_data/2018-11-22-events.csv', '/home/workspace/event_data/2018-11-16-events.csv', '/home/workspace/event_data/2018-11-26-events.csv', '/home/workspace/event_data/2018-11-24-events.csv', '/home/workspace/event_data/2018-11-29-events.csv', '/home/workspace/event_data/2018-11-15-events.csv', '/home/workspace/event_data/2018-11-20-events.csv', '/home/workspace/event_data/2018-11-06-events.csv', '/home/workspace/event_data/2018-11-18-events.csv', '/home/workspace/event_data/2018-11-21-events.csv', '/home/workspace/event_data/2018-11-10-events.csv', '/home/workspace/event_data/2018-11-23-events.csv', '/home/workspace/event_data/2018-11-02-events.c

#### Processing the files to create the data file csv that will be used for Apache Casssandra tables

In [12]:
# initiating an empty list of rows that will be generated from each file
full_data_rows_list = [] 
    
# for every filepath in the file path list 
for f in file_path_list:

# reading csv file 
    with open(f, 'r', encoding = 'utf8', newline='') as csvfile: 
        # creating a csv reader object 
        csvreader = csv.reader(csvfile) 
        next(csvreader)
        
 # extracting each data row one by one and append it        
        for line in csvreader:
            #print(line)
            full_data_rows_list.append(line) 
            
# uncomment the code below if you would like to get total number of rows 
print("number of rows:", len(full_data_rows_list))
# uncomment the code below if you would like to check to see what the list of event data rows will look like
#print(full_data_rows_list)

# creating a smaller event data csv file called event_datafile_full csv that will be used to insert data into the \
# Apache Cassandra tables
csv.register_dialect('myDialect', quoting=csv.QUOTE_ALL, skipinitialspace=True)

with open('event_datafile_new.csv', 'w', encoding = 'utf8', newline='') as f:
    writer = csv.writer(f, dialect='myDialect')
    writer.writerow(['artist','firstName','gender','itemInSession','lastName','length', 'level','location','sessionId','song','userId'])
    for row in full_data_rows_list:
        if (row[0] == ''):
            continue
        writer.writerow((row[0], row[2], row[3], row[4], row[5], row[6], row[7], row[8], row[12], row[13], row[16]))


number of rows: 8056


In [13]:
# check the number of rows in your csv file
with open('event_datafile_new.csv', 'r', encoding = 'utf8') as f:
    print(sum(1 for line in f))

6821


# Part II. Complete the Apache Cassandra coding portion of your project. 

#### Now you are ready to work with the CSV file titled <font color=red>event_datafile_new.csv</font>, located within the Workspace directory.  The event_datafile_new.csv contains the following columns: 
- artist 
- firstName of user
- gender of user
- item number in session
- last name of user
- length of the song
- level (paid or free song)
- location of the user
- sessionId
- song title
- userId

The image below is a screenshot of what the denormalized data should appear like in the <font color=red>**event_datafile_new.csv**</font> after the code above is run:<br>

<img src="images/image_event_datafile_new.jpg">

### Apache Cassandra code is in the cells below

##### Creating a Cluster

In [14]:
# This should make a connection to a Cassandra instance your local machine 
# (127.0.0.1)

from cassandra.cluster import Cluster
cluster = Cluster()

# To establish connection and begin executing queries, need a session
session = cluster.connect()

##### Create Keyspace

In [15]:
try:
    session.execute("""CREATE KEYSPACE IF NOT EXISTS sparkifydb
                    WITH REPLICATION = {'class':'SimpleStrategy', 'replication_factor':1}""")
except Exception as e:
    print(e)

##### Set Keyspace

In [16]:
try:
    session.set_keyspace('sparkifydb')
except Exception as e:
    print(e)

### Create queries to ask the following three questions of the data


In [17]:
from tqdm import tqdm # for progress bars
file = 'event_datafile_new.csv'

### 1. Give me the artist, song title and song's length in the music app history that was heard during  sessionId = ..., and itemInSession = ...

In [32]:
query1 = "SELECT artist, song, length FROM session_items WHERE sessionId=%s AND itemInSession=%s"

In [19]:
# We have provided part of the code to set up the CSV file. Please complete the Apache Cassandra code below

session.execute("""CREATE TABLE IF NOT EXISTS session_items 
                   (sessionId int, 
                   itemInSession int,
                   artist text, 
                   song text,
                   length float, 
                   PRIMARY KEY (sessionId, itemInSession))
                """)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in tqdm(csvreader):
        query = "INSERT INTO session_items (sessionId, itemInSession, artist, song, length) "
        query = query + "VALUES (%s, %s, %s, %s, %s)"
        
        session.execute(query, (int(line[8]), int(line[3]), line[0], line[9], float(line[5])))

6820it [00:12, 560.66it/s]


##### Do a SELECT to verify that the data have been inserted into each table

In [21]:
try:
    rows = session.execute(query1, (338, 4))
except Exception as e:
    print(e)
    
for row in rows:
    print("\t".join([f"{t}: {row.__getattribute__(t):25}" for t in rows.column_names]))

artist: Faithless                	song: Music Matters (Mark Knight Dub)	length:        495.30731201171875


### 2. Give me only the following: name of artist, song (sorted by itemInSession) and user (first and last name) for userid = ..., sessionid = ...

In [22]:
query2 = "SELECT artist, song, firstName, lastName  FROM user_session_playlists WHERE userId=%s AND sessionId=%s"   

In [23]:
# This table requires the partition be to be composite, 
# because a user's sessions might be distributed over nodes.
# itemInSession defines the ordering of the items within a user's session.
session.execute("""CREATE TABLE IF NOT EXISTS user_session_playlists 
                   (userId int,
                   sessionId int, 
                   itemInSession int, 
                   artist text, 
                   song text, 
                   firstName text, 
                   lastName text, 
                   PRIMARY KEY ((userId, sessionId), itemInSession))
                """)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in tqdm(csvreader):
        query = "INSERT INTO user_session_playlists (userId, sessionId, itemInSession, artist, song, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s, %s, %s, %s)"
               
        session.execute(query, (int(line[10]), int(line[8]), int(line[3]), line[0], line[9], line[1], line[4]))


6820it [00:10, 641.70it/s]


In [25]:
try:
    rows = session.execute(query2, (10, 182))
except Exception as e:
    print(e)
    
for row in rows:
    print("\t".join([f"{t}: {row.__getattribute__(t):25}" for t in rows.column_names]))

artist: Down To The Bone         	song: Keep On Keepin' On       	firstname: Sylvie                   	lastname: Cruz                     
artist: Three Drives             	song: Greece 2000              	firstname: Sylvie                   	lastname: Cruz                     
artist: Sebastien Tellier        	song: Kilometer                	firstname: Sylvie                   	lastname: Cruz                     
artist: Lonnie Gordon            	song: Catch You Baby (Steve Pitron & Max Sanna Radio Edit)	firstname: Sylvie                   	lastname: Cruz                     


### 3. Give me every user name (first and last) in my music app history who listened to the song ...

In [26]:
query3 = "SELECT firstName, lastName FROM song_users WHERE song=%s"

In [27]:
# This table requires a composite key because multiple people may listen to the same song.
# Combining the song name with the sessionId and the itemInSession number results in a unique composite key.
session.execute("""CREATE TABLE IF NOT EXISTS song_users 
                   (song text, 
                   userId int,  
                   firstName text, 
                   lastName text, 
                   PRIMARY KEY (song, userId))
                """)

with open(file, encoding = 'utf8') as f:
    csvreader = csv.reader(f)
    next(csvreader) # skip header
    for line in tqdm(csvreader):
        query = "INSERT INTO song_users (song, userId, firstName, lastName) "
        query = query + "VALUES (%s, %s, %s, %s)"
               
        session.execute(query, (line[9], int(line[10]), line[1], line[4]))

6820it [00:10, 658.84it/s]


In [29]:
try:
    rows = session.execute(query3, ('All Hands Against His Own',)) 
except Exception as e:
    print(e)
    
for row in rows:
    print("\t".join([f"{t}: {row.__getattribute__(t):15}" for t in rows.column_names]))

firstname: Jacqueline     	lastname: Lynch          
firstname: Tegan          	lastname: Levine         
firstname: Sara           	lastname: Johnson        


### Drop the tables before closing out the sessions

In [30]:
session.execute("DROP TABLE session_items")
session.execute("DROP TABLE user_session_playlists")
session.execute("DROP TABLE song_users")

### Close the session and cluster connection¶

In [31]:
session.shutdown()
cluster.shutdown()